In [2]:
import subprocess
import pandas as pd

In [10]:
model_name = 'models/22jul_no_TT_visitor'
model_data = 'models-data/data_30junB.dat'

In [11]:
p = subprocess.Popen(['C:/Program Files (x86)/biogeme-2.4/biogeme-2.4/biogeme.exe', model_name,model_data], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line),
retval = p.wait()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
biogeme 2.4 [Sun Oct 25 23:01:47 WEST 2015]
Michel Bierlaire, EPFL
-- Compiled by biogeme@hotmail.com on MINGW32_NT-6.1
See http://biogeme.epfl.ch
                    !! CFSQP is available !!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
	"In every non-trivial program there is at least one bug."


[09:42:41]patFileNames.cc:52  models/22jul_no_TT_visitor.par does not exist
[09:42:41]patFileNames.cc:56  Trying default.par instead
[09:42:41]patBiogeme.cc:134  Read default.par
 Opening file models-data/data_30junB.dat
 Data  file... line 500	Memory: 51 Kb
 Total obs.:   837
 Total memory: 86.7461 Kb
 Run time for data processing: 00:00
 Init loglike=-985.941
     gmax Iter   radius        f(x)     Status       rhok nFree
 +2.39e-001    1 1.00e+000 +9.8594093e+002 ****OutTrReg +9.77e-001 5  ++ P
 +1.61e-001    2 2.00e+000 +7.8221646e+002 ****Converg  +1.13e+000 5  ++ P
 +6.94e-002    3 4.00e+000 +6.2139824e+002 ****Converg  +1.

In [12]:
with open(model_name+'_res.mod','wb') as simmod:
    with open(model_name+'.res','rb') as res:
        searchlines = res.readlines()
        foundit = False
        for i, line in enumerate(searchlines):
            if foundit==False:
                if b'[SampleEnum]' in line:
                    print('Found it.')
                    foundit = True
                simmod.write(line)
            else:
                if line.decode("utf-8").find('0\r\n')==0:
                    print('Changed to 100.')
                    simmod.write(b'100\r\n')
                    foundit==False
                else:
                    simmod.write(line)

Found it.
Changed to 100.


In [13]:
p = subprocess.Popen(['C:/Program Files (x86)/biogeme-2.4/biogeme-2.4/biosim.exe', model_name+'_res',model_data], shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line),
retval = p.wait()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
biogeme 2.4 [Sun Oct 25 23:01:47 WEST 2015]
Michel Bierlaire, EPFL
-- Compiled by biogeme@hotmail.com on MINGW32_NT-6.1
See http://biogeme.epfl.ch
                    !! CFSQP is available !!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
	"In every non-trivial program there is at least one bug."


[09:43:50]patFileNames.cc:52  models/22jul_no_TT_visitor_res.par does not exist
[09:43:50]patFileNames.cc:56  Trying default.par instead
[09:43:50]patBiogeme.cc:134  Read default.par
 Opening file models-data/data_30junB.dat
 Data  file... line 500	Memory: 51 Kb
 Total obs.:   837
 Total memory: 86.7461 Kb
 59%	[500/838] Est. term. time: 07/22/16 09:43:50 (in 00:00)
 BIOSIM Input file
 Model specification:	models/22jul_no_TT_visitor_res.mod
 BIOSIM Output file
 Sample enumeration:	models/22jul_no_TT_visitor_res.enu


In [14]:
simres = pd.read_csv(model_name+'_res.enu',sep='\t')
simres.head()

,Choice_Id,P_choice,V_A1_TAXI,V_A2_TRANSIT,V_A3_OTHER,V_A4_FREE,P_A1_TAXI,Residual_A1_TAXI,P_A2_TRANSIT,Residual_A2_TRANSIT,...,Simul91,Simul92,Simul93,Simul94,Simul95,Simul96,Simul97,Simul98,Simul99,Simul100
0,1,0.520825,-0.428312,-1.293124,-1.12366,-9999.99,0.520825,0.479175,0.219335,-0.219335,...,3,2,1,3,1,1,2,1,3,1
1,1,0.520825,-0.428312,-1.293124,-1.12366,-9999.99,0.520825,0.479175,0.219335,-0.219335,...,1,1,1,1,1,1,1,1,3,3
2,1,0.520825,-0.428312,-1.293124,-1.12366,-9999.99,0.520825,0.479175,0.219335,-0.219335,...,3,3,1,2,1,2,3,1,3,1
3,1,0.520825,-0.428312,-1.293124,-1.12366,-9999.99,0.520825,0.479175,0.219335,-0.219335,...,1,1,2,1,2,2,3,2,2,3
4,1,0.520825,-0.428312,-1.293124,-1.12366,-9999.99,0.520825,0.479175,0.219335,-0.219335,...,2,2,1,2,1,1,1,1,2,1


In [15]:
simul_start = list(simres.columns).index('Simul1')
from collections import Counter
def count_correct(row):
    return list(row[simul_start:]).count(row.Choice_Id)
def simul_mode(row):
    return int(Counter(list(row[simul_start:])).most_common(1)[0][0])

In [16]:
simres['Accuracy'] = simres.apply(count_correct,axis=1)
simres['Mode_Prediction'] = simres.apply(simul_mode,axis=1)

In [17]:
simres['ModeAccurate'] = simres['Choice_Id']==simres['Mode_Prediction']

In [18]:
simres.groupby('Choice_Id')['Accuracy'].mean()

Choice_Id
1    57.957143
2    21.336134
3    23.443662
4    99.991189
Name: Accuracy, dtype: float64

In [19]:
simres.groupby('Choice_Id')['ModeAccurate'].mean()

Choice_Id
1    0.98000
2    0.07563
3    0.00000
4    1.00000
Name: ModeAccurate, dtype: float64

In [20]:
simres.groupby(['Choice_Id','Mode_Prediction']).size()

Choice_Id  Mode_Prediction
1          1                  343
           2                    7
2          1                  110
           2                    9
3          1                  137
           2                    5
4          4                  227
dtype: int64